In [2]:
## 기본 library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
## model library
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from keras import models, layers, optimizers
## helper library
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import accuracy_score
from keras.callbacks import EarlyStopping, ModelCheckpoint
import keras.backend as K
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from bayes_opt import BayesianOptimization

from sklearn.datasets import load_breast_cancer
from sklearn.cluster import KMeans

d = load_breast_cancer()

data = pd.DataFrame(d.data, columns=d.feature_names)
target = pd.DataFrame(d.target)



In [3]:
std = StandardScaler()
for c in data.columns:
    data[c] = std.fit_transform(data[c].values.reshape(-1, 1))
    
train_X, test_X, train_y, test_y = train_test_split(data, target, test_size=0.3, random_state=2019, shuffle=True)

In [4]:
train_X = train_X.reset_index().drop('index', axis=1)
train_y = train_y.reset_index().drop('index', axis=1)
test_X = test_X.reset_index().drop('index', axis=1)
test_y = test_y.reset_index().drop('index', axis=1)

In [5]:
km = KMeans(n_clusters=5).fit(train_X)
cl = pd.get_dummies(km.labels_, prefix='kmean')
train_X = pd.concat([train_X, cl], axis=1)

km = KMeans(n_clusters=5).fit(test_X)
cl = pd.get_dummies(km.labels_, prefix='kmean')
test_X = pd.concat([test_X, cl], axis=1)

In [6]:
kf = KFold(n_splits=5, shuffle=True, random_state=0)

In [7]:
lr = LogisticRegression(penalty='l2')

In [8]:
lr.fit(train_X, train_y)

c:\users\young\anaconda3\envs\study\lib\site-packages\sklearn\utils\validation.py:744: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [9]:
cross_val_score(lr, train_X, train_y, cv=kf)

c:\users\young\anaconda3\envs\study\lib\site-packages\sklearn\utils\validation.py:744: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\young\anaconda3\envs\study\lib\site-packages\sklearn\utils\validation.py:744: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\young\anaconda3\envs\study\lib\site-packages\sklearn\utils\validation.py:744: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\young\anaconda3\envs\study\lib\site-packages\sklearn\utils\validation.py:744: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Ple

array([0.975     , 0.925     , 1.        , 0.96202532, 0.98734177])

In [10]:
accuracy_score(test_y, lr.predict(test_X))

0.9707602339181286

In [11]:
input_layer = layers.Input(shape=((train_X.shape[1:])))

l1 = layers.Dense(32 ,activation='relu')(input_layer)
l2 = layers.Dense(16, activation='relu')(l1)

out_layer = layers.Dense(1, activation='sigmoid')(l2)

model = models.Model(input_layer, out_layer)

model.compile(optimizer = 'adam',
             loss = 'binary_crossentropy',
             metrics = ['acc'])

In [12]:
model.fit(train_X, train_y, 
         epochs=10,
         validation_split=0.3)

Train on 278 samples, validate on 120 samples
Epoch 1/10
278/278 [==============================] - 0s 1ms/step - loss: 0.6929 - acc: 0.5504 - val_loss: 0.5522 - val_acc: 0.8250
Epoch 2/10
278/278 [==============================] - 0s 116us/step - loss: 0.4627 - acc: 0.8597 - val_loss: 0.3792 - val_acc: 0.9333
Epoch 3/10
278/278 [==============================] - 0s 118us/step - loss: 0.3344 - acc: 0.9137 - val_loss: 0.2884 - val_acc: 0.9417
Epoch 4/10
278/278 [==============================] - 0s 122us/step - loss: 0.2623 - acc: 0.9388 - val_loss: 0.2385 - val_acc: 0.9417
Epoch 5/10
278/278 [==============================] - 0s 115us/step - loss: 0.2167 - acc: 0.9604 - val_loss: 0.2083 - val_acc: 0.9417
Epoch 6/10
278/278 [==============================] - 0s 114us/step - loss: 0.1863 - acc: 0.9604 - val_loss: 0.1867 - val_acc: 0.9417
Epoch 7/10
278/278 [==============================] - 0s 111us/step - loss: 0.1624 - acc: 0.9604 - val_loss: 0.1716 - val_acc: 0.9417
Epoch 8/10
278/278

In [13]:
model.evaluate(test_X, test_y)

171/171 [==============================] - 0s 52us/step


[0.14942890989501573, 0.9649122953414917]